# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [3]:
#Run this cell once to install all the unavailable packages.
!python -m pip install requests
!python -m pip install urlparse
!python -m pip install nltk

You should consider upgrading via the 'c:\Users\ASUS\AppData\Local\Programs\Python\Python397\python.exe -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement urlparse (from versions: none)
ERROR: No matching distribution found for urlparse
You should consider upgrading via the 'c:\Users\ASUS\AppData\Local\Programs\Python\Python397\python.exe -m pip install --upgrade pip' command.


  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached click-8.1.6-py3-none-any.whl (97 kB)
  Using cached joblib-1.3.1-py3-none-any.whl (301 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\ASUS\AppData\Local\Programs\Python\Python397\python.exe -m pip install --upgrade pip' command.


In [5]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
from nltk import ngrams  
from collections import Counter, defaultdict
  
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [33]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.ngram_dictionary = self.build_ngrams_dictionary_by_length(n=5)
        self.current_dictionary = []

        self.ngram_weight_map = {5: 0.33 , 4: 0.267, 3 : 0.2, 2: 0.1333, 1:0.066}
        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def build_ngrams_dictionary(self, dictionary, n):  
        ngram_dict = {}  
        for i in range(1, n+1):  
            ngram_list = [word[j:j+i] for word in dictionary for j in range(len(word)-i+1)]  
            ngram_counter = Counter(ngram_list)  
            ngram_dict[i] = dict(ngram_counter)  
        return ngram_dict  

    def build_ngrams_dictionary_by_length(self, n):

        dictionary_by_length = defaultdict(list)
        ngram_dictionary_by_length = defaultdict(dict)

        for word in self.full_dictionary:
            dictionary_by_length[len(word)].append(word)
        
        for key, value in dictionary_by_length.items():
            ngram_dictionary_by_length[key] = self.build_ngrams_dictionary(value, n)
            
        return ngram_dictionary_by_length

    def guess(self, word): # word input example: "_ p p _ e "

        clean_word = word[::2].replace("_",".")
        len_word = len(clean_word)

        #For each blank position, find the probability of each letter to be in that position.  
        letter_probs = defaultdict(lambda: defaultdict(float))  
        total_counts = defaultdict(float)  
        final_letter_probs = defaultdict(float)

        #update the current dictionary whenever we make an incorrect guess
        if len(self.guessed_letters) > 0 and self.guessed_letters[-1] not in clean_word:
            new_dictionary = []

            # iterate through all of the words in the old plausible dictionary
            for dict_word in self.current_dictionary:
                # continue if the word is not of the appropriate length
                if len(dict_word) != len_word:
                    continue

                search_space_in_word = "".join([letter for idx, letter in enumerate(dict_word) if clean_word[idx] == '.'])
                if bool(re.search(self.guessed_letters[-1], search_space_in_word)):
                    continue 

                new_dictionary.append(dict_word)
            self.current_dictionary = new_dictionary
            current_dictionary = self.build_ngrams_dictionary(new_dictionary, n=5)
            self.ngram_dictionary[len_word] = current_dictionary
        
        else:
            current_dictionary = self.ngram_dictionary[len_word]

        
        #Uncomment this to go back to working code.
        #current_dictionary = self.ngram_dictionary[len_word]

        for ngram_num in range(1, 6):
            for j in range(len(clean_word)-ngram_num+1):
                ngram = clean_word[j:j+ngram_num]

                for w in current_dictionary[ngram_num]:  
                    if not re.match(ngram,w):
                        continue
                    for i, letter in enumerate(w):
                        if letter in self.guessed_letters:
                            continue  
                        if ngram[i] == '.':  
                            letter_probs[ngram_num][letter] += current_dictionary[ngram_num][w]
                            total_counts[ngram_num] += current_dictionary[ngram_num][w]  
            
            # Using this, we will get the probabilities of each letter in each position.  
              
            for letter in letter_probs[ngram_num]:  
                letter_probs[ngram_num][letter] /= total_counts[ngram_num] 
                final_letter_probs[letter] += self.ngram_weight_map[ngram_num] * letter_probs[ngram_num][letter]
        
                    
        #print(f'Final Letter probs = {final_letter_probs}')  
        guess_letter = '!'
            
        # Whichever letter's probability is highest, that will be the letter to be suggested.  
        max_prob = 0  
        guess_letter = '!'  
        for letter, prob in final_letter_probs.items(): 
            if prob > max_prob and letter not in self.guessed_letters:  
                max_prob = prob  
                guess_letter = letter 

        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = sorted(self.ngram_dictionary[len_word][1].items(), lambda x: x[1], reverse=True)
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break              
        
        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [34]:
api = HangmanAPI(access_token="2ddac8e65fab56dbe06a100478924b", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [29]:
for i in range(100):
    print('Playing ', i, ' th game')
    api.start_game(practice=1,verbose=True)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Successfully start a new game! Game ID: b22f90e08d11. # of tries remaining: 6. Word: _ _ _ .
Final Letter probs = defaultdict(<class 'float'>, {'a': 0.02858549144328336, 'g': 0.011407693472663941, 'l': 0.015205209753142511, 'm': 0.018158412842647284, 'o': 0.019733454490383162, 'r': 0.016855974557019537, 's': 0.023595335453581705, 'b': 0.013974708465848857, 'c': 0.02097152809329093, 'e': 0.02073678630925337, 'i': 0.019002726033621083, 'n': 0.012929728911101016, 'u': 0.013327275480842043, 'd': 0.017779797061941542, 'f': 0.01181659851582614, 'p': 0.017560199909132216, 'w': 0.007731334242011208, 'v': 0.00753824019385128, 'q': 0.0023777071028320464, 't': 0.018461305467211874, 'h': 0.009965167348175073, 'j': 0.0033507496592457974, 'k': 0.006057852491291837, 'y': 0.007099045888232621, 'z': 0.002025594426775708, 'x': 0.003752082386793882})
Guessing letter: a
Sever response: {'game_id': 'b22f90e08d11', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ '}
Final Letter p

In [30]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


run 1022 practice games out of an allotted 100,000. practice success rate so far = 0.235


## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [38]:
for i in range(366):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Playing  1  th game
Playing  2  th game
Playing  3  th game
Playing  4  th game
Playing  5  th game
Playing  6  th game
Playing  7  th game
Playing  8  th game
Playing  9  th game
Playing  10  th game
Playing  11  th game
Playing  12  th game
Playing  13  th game
Playing  14  th game
Playing  15  th game
Playing  16  th game
Playing  17  th game
Playing  18  th game
Playing  19  th game
Playing  20  th game
Playing  21  th game
Playing  22  th game
Playing  23  th game
Playing  24  th game
Playing  25  th game
Playing  26  th game
Playing  27  th game
Playing  28  th game
Playing  29  th game
Playing  30  th game
Playing  31  th game
Playing  32  th game
Playing  33  th game
Playing  34  th game
Playing  35  th game
Playing  36  th game
Playing  37  th game
Playing  38  th game
Playing  39  th game
Playing  40  th game
Playing  41  th game
Playing  42  th game
Playing  43  th game
Playing  44  th game
Playing  45  th game
Playing  46  th game
Playing  47  th game
Pl

HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [39]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.449


In [40]:
total_recorded_runs

1000